In [62]:
from pulp import *
import pulp
import pandas as pd
import numpy as np
import random

In [63]:
# Read the Excel file into a DataFrame
df = pd.read_excel('file.xlsx')
# results = pd.read_excel('./results.xlsx')
# dimension = pd.read_excel('./dataset/product_hierarchy_planning.xlsx')

# Convert the DataFrame to a NumPy array
data = df.values
# dataResults = results.values
# dataDimensions = dimension.values

products = []

# for i in range(len(dataResults)):
#     predictedProfits = dataResults[i][6]
#     predictedProductQuantities = dataResults[i][5]
#     dimension = dataDimensions[i][]
#     productName = dataResults[i][0]

for row in data:
    predictedProfits = row[0]
    predictedProductQuantities = row[1]
    dimension = row[2]
    productName = row[3]

    product = {'predictedProfits': predictedProfits, 'predictedProductQuantities': predictedProductQuantities, 'dimension': dimension, 'productName': productName}
    products.append(product)

print(products)

[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1'}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2'}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3'}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4'}]


In [64]:
def calculate_profit(products):
    for product in products:
        profit = product['predictedProfits'] / product['predictedProductQuantities']
        product['productProfit'] = profit
    return products

In [65]:
productTest = calculate_profit(products)
print(productTest)

[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2', 'productProfit': 10.0}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3', 'productProfit': 10.0}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4', 'productProfit': 10.0}]


In [5]:
# Creation of the ILP model 
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)

In [6]:
# Define the decision variables
decisionVariables = []
def createDictionaryWithDecisionVariable():
    for product in products:
        decisionVariable = {product['productName']:  pulp.LpVariable(product['productName'], lowBound=0, cat='Integer')}
        decisionVariables.append(decisionVariable)
        
createDictionaryWithDecisionVariable()
print(decisionVariables)

[{'p1': p1}, {'p2': p2}, {'p3': p3}, {'p4': p4}]


In [7]:
# function to calculate objective function

def objectiveFunction(products):
    model = 0
    count = 0
    for product in products:
        model = model + (product['predictedProfits'] * decisionVariables[count][product['productName']])
        count = count + 1
    return model

#objective function
model += objectiveFunction(products)
print(model)

ILP_Problem:
MAXIMIZE
100*p1 + 200*p2 + 300*p3 + 10*p4 + 0
VARIABLES
0 <= p1 Integer
0 <= p2 Integer
0 <= p3 Integer
0 <= p4 Integer



In [66]:
#add user input for the multiple variables

minTotalAmount = int(input("Please insert minimum total amount of products: "))
maxTotalAmount = int(input("Please insert maximum  total amount of products: "))

#create two list with min and max amount per product by user insert

minAmountProduct = int(input("Please insert minimum  amount of products : "))
maxAmountProduct  = int(input("Please insert maximum  amount of products : "))

minProductVariety = int(input("Please insert minimum number of product variety: "))
maxProductVariety  = int(input("Please insert maximum  number of product variety: "))

storeDimension  = int(input("Please insert store dimension: "))

In [ ]:
# Define the constraints

 # function to calculate variety of products
def get_variety(products):
    # Create an empty set to store the product names
    product_set = []
    
    # Iterate over the products
    for product in products:
        # Get the product name and add it to the set
        product_name = product['productName']
        product_set.append(product_name)
        
    # Return the length of the set
    return len(product_set)

print(get_variety(products))
variety=get_variety(products)

#see why it results in Infeasible
# model += variety <= maxProductVariety
# model += variety >= minProductVariety


#restriction to total amount of products

def maxMinTotalProductsRestriction(products):
    restrictFunction = 0
    count = 0
    for product in products:
        restrictFunction = restrictFunction + decisionVariables[count][product['productName']]
        count = count + 1
    return restrictFunction
    
model += maxMinTotalProductsRestriction(products) <= maxTotalAmount #restricao de quantidade maximo total de produtos
model += maxMinTotalProductsRestriction(products) >= minTotalAmount #restricao de valor minimo total de produtos

# variety of products

#restriction to maximum quantity of products
def maxMinQuantityProductsRestriction(products,model):
    count = 0
    for product in products:
        model += decisionVariables[count][product['productName']] <= maxAmountProduct
        model += decisionVariables[count][product['productName']] >= minAmountProduct
        count = count + 1
    
maxMinQuantityProductsRestriction(products, model)

print(maxAmountProduct)
print(minAmountProduct)


def maxDimensionProductsRestriction(products):
    count = 0
    restrictFunction = 0
    for product in products:                            
        restrictFunction = restrictFunction + (product['dimension'] * decisionVariables[count][product['productName']])
        count = count +1
    return restrictFunction

print(maxDimensionProductsRestriction(products))
#restricao de dimensões de produtos, menor que dimensao todal da loja
model += maxDimensionProductsRestriction(products) <= storeDimension 


4
10
1
5*p1 + 2*p2 + p3 + 20*p4


In [ ]:
print(model)

ILP_Problem:
MAXIMIZE
100*p1 + 200*p2 + 300*p3 + 10*p4 + 0
SUBJECT TO
_C1: p1 + p2 + p3 + p4 <= 100

_C2: p1 + p2 + p3 + p4 >= 10

_C3: p1 <= 10

_C4: p1 >= 1

_C5: p2 <= 10

_C6: p2 >= 1

_C7: p3 <= 10

_C8: p3 >= 1

_C9: p4 <= 10

_C10: p4 >= 1

_C11: 5 p1 + 2 p2 + p3 + 20 p4 <= 1.11111111111e+21

VARIABLES
0 <= p1 Integer
0 <= p2 Integer
0 <= p3 Integer
0 <= p4 Integer



In [ ]:
# Solve the problem
status = model.solve()

In [ ]:
print(pulp.LpStatus[status])

Optimal


In [ ]:
def printValuesReturned():
    for v in model.variables():
        try:
            print(v.name,"=", v.value())
        except:
            print("error couldnt find value")
            
printValuesReturned()
print(len(products))
print(products)
print(products[0]['dimension'])

p1 = 10
p2 = 10
p3 = 10
p4 = 10
4
[{'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5, 'productName': 'p1', 'productProfit': 10.0}, {'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2, 'productName': 'p2', 'productProfit': 10.0}, {'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1, 'productName': 'p3', 'productProfit': 10.0}, {'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20, 'productName': 'p4', 'productProfit': 10.0}]
5


In [56]:



# Functions to validate restrictions
# return 1 if restriction is repected else 0

#Total Amount Restriction
def validateTotalAmount(individuo):
    somaTotalAmountIndividuo = sum(individuo)
    if (somaTotalAmountIndividuo >= minTotalAmount) and (somaTotalAmountIndividuo <= maxTotalAmount):
        return 1
    else:
        return 0
    
#Total Quantity Restriction
def validateTotalQuantity(individuo):
    contador = 0
    individuoLen = len(individuo)
    for indAux in individuo:
        if (indAux >= minAmountProduct) and (indAux <= maxAmountProduct):
            contador = contador + 1
    if(contador == individuoLen):
        return 1
    else:
        return 0

#Dimension Restriction
def validateDimension(individuo, products):
    totalDimension = 0
    contador = 0
    for indAux in individuo:
        totalDimension = indAux * products[contador]['dimension']
        contador = contador + 1
    if(totalDimension > storeDimension):
        return 0
    else:
        return 1


#-------------------------------------------------------------------------------------------------------------------
# Create First Population

def firstpopulation(population, products, populationSize):

    while len(population) < populationSize:
        solution = []
        totalQuantity = 0
        totalDimension = 0
        
        #generates the chromossomes each representing one product
        for j in range(len(products)):
            
            quantity = random.randint(0, round((maxAmountProduct/3)))
            solution.append(quantity)
            
        #Total Amount Restriction
        totalAmountRestriction = validateTotalAmount(solution)
        #Total Quantity Restriction
        totalQuantityRestriction = validateTotalQuantity(solution)
        #Dimension Restriction
        dimensionRestriction = validateDimension(solution, products)
        
        if (totalAmountRestriction == 1) and (totalQuantityRestriction == 1) and (dimensionRestriction == 1):
            population.append(solution)
        
    
    return population


# Create Fitness function to give the best solution-HUGO

def createFitnessFunction(population, products, finalPopulation, taxNoCrossover):

    arraySomaProfits = []
    arrayFitProfitAndQuantity = []

    for person in population:

        soma = 0
        contador = 0

        for productQuantity in person:

            soma = soma + (productQuantity * products[contador]['predictedProfits'])
            contador = contador + 1

        auxVar = {'FitProfit': soma, 'FitQuantities': person}
        arrayFitProfitAndQuantity.append(auxVar)
        arraySomaProfits.append(soma)

    array = np.array(arraySomaProfits)
    array[::-1].sort()

    numberElementsNoCrossover = round(len(population)*taxNoCrossover)
    
    arrayFitProfitAndQuantity.sort(key=lambda x: x['FitProfit'], reverse=True)
    
    finalPopulationAux = [d['FitQuantities'] for d in arrayFitProfitAndQuantity[:numberElementsNoCrossover]]
    finalPopulation =finalPopulationAux[:]
    
    numberElementsToPop = len(population) - numberElementsNoCrossover
    print("Number elements: ", numberElementsToPop)
    arrayFitProfitAndQuantityAux =  arrayFitProfitAndQuantity[::-1]
    populationAux = [d['FitQuantities'] for d in arrayFitProfitAndQuantityAux[:numberElementsToPop]]
    print("Len pop aux:", len(populationAux))
    print("pop aux:", populationAux)
    population = []
    population = populationAux[:]
    print("Final Pop Fit:", population)
    print("Len Pop Fit:", len(population))
    
        
    return finalPopulation


# Function to crossover

def crossover(population, finalPopulation):
    
    stopCrossover = 0
    
    while len(population)>0:
        print(len(population))
        if len(population) == 1:
            
            finalPopulation.append(population[0])
            population.remove(population[0])
            
        else:
            
            indexFirstPai = random.randint(0, len(population)-1)
            indexSecondPai = random.randint(0, len(population)-1)
            
            while indexFirstPai == indexSecondPai:
                indexSecondPai = random.randint(0, len(population)-1)
            
            
            firstPai = population[indexFirstPai]
            secondPai = population[indexSecondPai]
            
            firstPaiToDelete = population[indexFirstPai]
            secondPaiToDelete = population[indexSecondPai]
            
            crossoverPoint = random.randint(0, len(population[indexFirstPai])-1)
            
            for i in range(crossoverPoint):
                aux = firstPai[i]
                firstPai[i] = secondPai[i]
                secondPai[i] = aux

             #validate that first chield is good
            totalAmountRestriction1 = validateTotalAmount(firstPai)
            totalQuantityRestriction1 = validateTotalQuantity(firstPai)
            dimensionRestriction1 = validateDimension(firstPai, products)
            #validate that second chield is good
            totalAmountRestriction2 = validateTotalAmount(secondPai)
            totalQuantityRestriction2 = validateTotalQuantity(secondPai)
            dimensionRestriction2 = validateDimension(secondPai, products)
                  
            if (totalAmountRestriction1 == 1) and (totalQuantityRestriction1 == 1) and (dimensionRestriction1 == 1) and (totalAmountRestriction2 == 1) and (totalQuantityRestriction2 == 1) and (dimensionRestriction2 == 1):
                population.remove(firstPaiToDelete)
                population.remove(secondPaiToDelete)
                finalPopulation.append(firstPai)
                finalPopulation.append(secondPai)
            
            # stopCrossover+= 1
            # print(stopCrossover)
            # if(stopCrossover == 10):
                
            #     population.remove(firstPaiToDelete)
            #     population.remove(secondPaiToDelete)
            #     finalPopulation.append(firstPaiToDelete)
            #     finalPopulation.append(secondPaiToDelete)

def mutation(finalPopulation, taxMutation, changeGene):
    
    mutationList=finalPopulation[:]
    
    for x in mutationList:
        t=random.randint(0, 100)
        
        if t >= taxMutation:
            mutationList.remove(x)

    for el in finalPopulation:
        
        initialEl=el
        
        for l in mutationList:
            
            if el==l:
                index=random.randint(0, len(mutationList[0])-1)
                
                randomSumOrLess = random.randint(0, 1)
                
                if(randomSumOrLess == 1):
                    el[index] = el[index] + round(el[index]*changeGene)
                else:
                    el[index] = el[index] - round(el[index]*changeGene)
                
                        
                totalAmountRestriction = validateTotalAmount(el)
                totalQuantityRestriction = validateTotalQuantity(el)
                dimensionRestriction = validateDimension(el, products)
                        
                if (totalAmountRestriction == 0) or (totalQuantityRestriction == 0) or (dimensionRestriction == 0):
                    
                    el=initialEl

In [90]:
# Functions to validate restrictions
# return 1 if restriction is repected else 0

#Total Amount Restriction
def validateTotalAmount(individuo):
    somaTotalAmountIndividuo = sum(individuo)
    if (somaTotalAmountIndividuo >= minTotalAmount) and (somaTotalAmountIndividuo <= maxTotalAmount):
        return 1
    else:
        return 0
    
#Total Quantity Restriction
def validateTotalQuantity(individuo):
    contador = 0
    individuoLen = len(individuo)
    for indAux in individuo:
        if (indAux >= minAmountProduct) and (indAux <= maxAmountProduct):
            contador = contador + 1
    if(contador == individuoLen):
        return 1
    else:
        return 0

#Dimension Restriction
def validateDimension(individuo, products):
    totalDimension = 0
    contador = 0
    for indAux in individuo:
        totalDimension = indAux * products[contador]['dimension']
        contador = contador + 1
    if(totalDimension > storeDimension):
        return 0
    else:
        return 1


#-------------------------------------------------------------------------------------------------------------------
# Create First Population

def firstpopulation(population, products, populationSize):

    while len(population) < populationSize:
        solution = []
        totalQuantity = 0
        totalDimension = 0
        
        #generates the chromossomes each representing one product
        for j in range(len(products)):
            
            quantity = random.randint(0, round((maxAmountProduct/3)))
            solution.append(quantity)
            
        #Total Amount Restriction
        totalAmountRestriction = validateTotalAmount(solution)
        #Total Quantity Restriction
        totalQuantityRestriction = validateTotalQuantity(solution)
        #Dimension Restriction
        dimensionRestriction = validateDimension(solution, products)
        
        if (totalAmountRestriction == 1) and (totalQuantityRestriction == 1) and (dimensionRestriction == 1):
            population.append(solution)
        
    
    return population

def createFitnessFunction(iterationsNumber, population, products, finalPopulation, taxNoCrossover, taxMutation, changeGene):
    iter = 0
    while iter < iterationsNumber:
            
        arraySomaProfits = []
        arrayFitProfitAndQuantity = []
        finalPopulation=[]
        
        for person in population:

            soma = 0
            contador = 0

            for productQuantity in person:

                soma = soma + (productQuantity * products[contador]['predictedProfits'])
                contador = contador + 1

            auxVar = {'FitProfit': soma, 'FitQuantities': person}
            arrayFitProfitAndQuantity.append(auxVar)
            arraySomaProfits.append(soma)

        array = np.array(arraySomaProfits)
        array[::-1].sort()

        numberElementsNoCrossover = round(len(population)*taxNoCrossover)
        
        arrayFitProfitAndQuantity.sort(key=lambda x: x['FitProfit'], reverse=True)
        
        finalPopulationAux = [d['FitQuantities'] for d in arrayFitProfitAndQuantity[:numberElementsNoCrossover]]
        finalPopulation =finalPopulationAux[:]
        
        numberElementsToPop = len(population) - numberElementsNoCrossover
        arrayFitProfitAndQuantityAux =  arrayFitProfitAndQuantity[::-1]
        populationAux = [d['FitQuantities'] for d in arrayFitProfitAndQuantityAux[:numberElementsToPop]]
        population = []
        population = populationAux[:]
        
            
        stopCrossover = 0
        print("Pop To Cross:", len(population))
        
        while len(population)>0:
            
            if len(population) == 1:
                
                finalPopulation.append(population[0])
                population.remove(population[0])
                
            else:
                
                indexFirstPai = random.randint(0, len(population)-1)
                indexSecondPai = random.randint(0, len(population)-1)
                
                while indexFirstPai == indexSecondPai:
                    indexSecondPai = random.randint(0, len(population)-1)
                
                
                firstPai = population[indexFirstPai]
                secondPai = population[indexSecondPai]
                
                firstPaiToDelete = population[indexFirstPai]
                secondPaiToDelete = population[indexSecondPai]
                
                crossoverPoint = random.randint(0, len(population[indexFirstPai])-1)
                
                for i in range(crossoverPoint):
                    aux = firstPai[i]
                    firstPai[i] = secondPai[i]
                    secondPai[i] = aux

                #validate that first chield is good
                totalAmountRestriction1 = validateTotalAmount(firstPai)
                totalQuantityRestriction1 = validateTotalQuantity(firstPai)
                dimensionRestriction1 = validateDimension(firstPai, products)
                #validate that second chield is good
                totalAmountRestriction2 = validateTotalAmount(secondPai)
                totalQuantityRestriction2 = validateTotalQuantity(secondPai)
                dimensionRestriction2 = validateDimension(secondPai, products)
                    
                if (totalAmountRestriction1 == 1) and (totalQuantityRestriction1 == 1) and (dimensionRestriction1 == 1) and (totalAmountRestriction2 == 1) and (totalQuantityRestriction2 == 1) and (dimensionRestriction2 == 1):
                    population.remove(firstPaiToDelete)
                    population.remove(secondPaiToDelete)
                    finalPopulation.append(firstPai)
                    finalPopulation.append(secondPai)
                
                # stopCrossover+= 1
                # print(stopCrossover)
                # if(stopCrossover == 10):
                    
                #     population.remove(firstPaiToDelete)
                #     population.remove(secondPaiToDelete)
                #     finalPopulation.append(firstPaiToDelete)
                #     finalPopulation.append(secondPaiToDelete)
                
        mutationList=finalPopulation[:]
        print("Pop With Cross:", len(mutationList))
        
        for x in mutationList:
            t=random.randint(0, 100)
            
            if t >= taxMutation:
                mutationList.remove(x)

        for el in finalPopulation:
            
            initialEl=el
            
            for l in mutationList:
                
                if el==l:
                    index=random.randint(0, len(mutationList[0])-1)
                    # print("Index para alterar: ", index)
                    randomSumOrLess = random.randint(0, 1)
                    # print("Random ou soma ou subtrai: ", randomSumOrLess)
                    # print("Nao Alterado: ", el)
                    if(randomSumOrLess == 1):
                        el[index] = el[index] + round(el[index]*changeGene)
                    else:
                        el[index] = el[index] - round(el[index]*changeGene)
                    
                    # print("Alterado: ", el)  
                    totalAmountRestriction = validateTotalAmount(el)
                    totalQuantityRestriction = validateTotalQuantity(el)
                    dimensionRestriction = validateDimension(el, products)
                            
                    if (totalAmountRestriction == 0) or (totalQuantityRestriction == 0) or (dimensionRestriction == 0):
                        
                        el=initialEl
        print("-----------------------------------------------------------------")
        print("")
        print("")
        print("")
        print("")
        print("")
        print("")
        print("")
        print("")
        print("")
        print("")
        print("")
        print("")          
        population=finalPopulation[:]
        print("Pop With Mut:", population)
        iter += 1
    return finalPopulation

In [81]:
population = []
finalPopulation = []
pop = []
population = firstpopulation(population, products, 100)
pop = createFitnessFunction(100, population, products, finalPopulation, 0.10, 5, 0.05)

Pop To Cross: [[10, 10, 10, 16], [10, 10, 12, 12], [10, 10, 12, 17], [17, 10, 10, 16], [16, 11, 10, 11], [13, 11, 11, 16], [16, 10, 11, 16], [14, 10, 12, 16], [12, 10, 13, 10], [14, 12, 11, 13], [12, 10, 13, 15], [11, 15, 10, 17], [14, 14, 10, 11], [11, 11, 13, 11], [13, 13, 11, 16], [16, 13, 10, 16], [17, 11, 11, 16], [10, 16, 10, 16], [11, 16, 10, 11], [11, 10, 14, 16], [13, 14, 11, 11], [10, 11, 14, 12], [13, 14, 11, 14], [12, 13, 12, 15], [11, 15, 11, 17], [13, 13, 12, 10], [15, 15, 10, 10], [11, 14, 12, 12], [14, 13, 12, 12], [11, 16, 11, 12], [16, 12, 12, 13], [17, 10, 13, 15], [17, 15, 10, 12], [17, 12, 12, 14], [12, 10, 15, 17], [12, 16, 11, 17], [13, 17, 10, 17], [12, 15, 12, 10], [12, 12, 14, 11], [16, 10, 14, 12], [14, 17, 10, 15], [11, 17, 11, 15], [14, 10, 15, 11], [15, 11, 14, 16], [10, 11, 16, 12], [15, 16, 11, 12], [13, 17, 11, 13], [12, 12, 15, 12], [10, 16, 13, 17], [11, 10, 17, 10], [13, 12, 15, 11], [10, 15, 14, 13], [12, 14, 14, 13], [15, 10, 16, 12], [10, 14, 15, 

In [94]:

def runNTimesGeneticAlgorithm(n):
    
    contador = 0
    
    while contador < n:
        
        population = []
        finalPopulation = []
        pop = []
        population = firstpopulation(population, products, 100)
        
        iterationsNumber = random.randint(0, 100)
        print(iterationsNumber)
        populationSize = random.randint(1,100)
        print(populationSize)
        taxNoCrossover = round(random.uniform(0.01, 0.10),2)
        print(taxNoCrossover)
        taxMutation = random.randint(1,5)
        print(taxMutation)
        changeGene = round(random.uniform(0.01,0.1),2)
        print(changeGene)
        
        pop = createFitnessFunction(iterationsNumber, population, products, finalPopulation, taxNoCrossover, taxMutation, changeGene)
        print("Cont: ", contador)
        print("Final Pop: ", pop)
        contador+= 1

In [97]:
runNTimesGeneticAlgorithm(1)

43
8
0.03
4
0.05
Cont:  0
Final Pop:  [[18, 14, 34, 18], [18, 14, 34, 18], [18, 14, 34, 18], [14, 13, 10, 11], [15, 16, 10, 15], [18, 13, 15, 10], [21, 13, 19, 12], [11, 14, 10, 13], [11, 17, 10, 20], [10, 12, 14, 16], [14, 10, 16, 13], [11, 10, 15, 10], [13, 10, 12, 13], [14, 14, 20, 10], [10, 17, 20, 15], [10, 13, 10, 10], [19, 16, 13, 19], [18, 17, 15, 14], [19, 10, 16, 22], [14, 17, 12, 15], [12, 18, 14, 14], [11, 10, 14, 16], [18, 13, 15, 10], [16, 10, 16, 14], [14, 10, 17, 13], [13, 10, 17, 16], [15, 14, 15, 19], [12, 10, 21, 10], [10, 15, 17, 17], [10, 12, 17, 10], [10, 11, 16, 10], [18, 14, 14, 13], [20, 17, 16, 14], [12, 17, 19, 21], [12, 10, 10, 12], [15, 11, 10, 17], [19, 10, 19, 18], [17, 18, 14, 10], [21, 14, 10, 14], [11, 16, 10, 13], [10, 18, 18, 20], [14, 19, 12, 15], [10, 17, 16, 16], [14, 14, 16, 10], [12, 15, 10, 10], [12, 10, 11, 10], [14, 16, 19, 17], [17, 10, 12, 10], [10, 10, 16, 10], [10, 13, 10, 10], [18, 14, 13, 10], [11, 17, 10, 20], [10, 12, 14, 10], [19, 10

In [46]:
pop = []

def geneticAlgorithm(iterationsNumber, populationSize, taxNoCrossover, taxMutation, changeGene):
    
    population = []
    finalPopulation = []
    population = firstpopulation(population, products, populationSize)
    for person in population:
        s = 0
        c = 0
        for productQuantity in person:
                s = s + (productQuantity * products[c]['predictedProfits'])
                c = c + 1
    print(s)
    print(population)
    
    i=0
    while i<iterationsNumber:
            
        finalPopulation=[]
        
        finalPopulation = createFitnessFunction(population, products, finalPopulation, taxNoCrossover)
        print("After Fitness:", len(finalPopulation), "Pop Len:", len(population))
        crossover(population, finalPopulation)
        print("After crossover:", len(finalPopulation), "Pop Len:", len(population))
        mutation(finalPopulation, taxMutation, changeGene)
        print("After mutation:", len(finalPopulation), "Pop Len:", len(population))
        population=finalPopulation[:]
        i+=1
        
    print("Acabou:", finalPopulation)
    for person in finalPopulation:
        soma = 0
        contador = 0
        for productQuantity in person:
                soma = soma + (productQuantity * products[contador]['predictedProfits'])
                contador = contador + 1
    print(soma)
    print(len(finalPopulation))

In [25]:
def runNTimesGeneticAlgorithm(n):
    
    contador = 0
    
    while contador < n:
        
        iterationsNumber = random.randint(0, 100)
        print(iterationsNumber)
        populationSize = random.randint(1,100)
        print(populationSize)
        taxNoCrossover = round(random.uniform(0, 0.10),2)
        print(taxNoCrossover)
        taxMutation = random.randint(0,5)
        print(taxMutation)
        changeGene = round(random.uniform(0,1),2)
        print(changeGene)
        
        geneticAlgorithm(iterationsNumber, populationSize, taxNoCrossover, taxMutation, changeGene)
        contador+= 1

In [57]:
geneticAlgorithm(500, 100, 0.10, 5, 0.01)

4100
[[14, 5, 11, 16], [10, 8, 15, 4], [10, 11, 7, 7], [14, 17, 8, 3], [12, 6, 12, 10], [11, 11, 17, 11], [13, 10, 13, 17], [5, 14, 16, 4], [17, 5, 16, 5], [12, 7, 9, 14], [16, 12, 3, 3], [17, 5, 4, 10], [12, 16, 14, 9], [13, 6, 13, 10], [15, 17, 8, 6], [4, 17, 6, 3], [11, 15, 11, 12], [12, 4, 8, 16], [5, 7, 15, 9], [15, 7, 8, 11], [9, 13, 11, 6], [14, 9, 4, 14], [5, 3, 5, 6], [6, 16, 4, 4], [13, 7, 4, 7], [16, 9, 7, 17], [9, 12, 3, 8], [4, 13, 11, 3], [17, 10, 15, 13], [15, 16, 17, 13], [10, 8, 12, 10], [13, 11, 15, 3], [16, 4, 10, 11], [4, 16, 17, 9], [5, 6, 13, 16], [11, 8, 12, 14], [16, 16, 13, 6], [4, 9, 13, 8], [14, 11, 9, 10], [9, 4, 12, 6], [14, 6, 10, 5], [16, 3, 13, 3], [9, 14, 5, 17], [3, 15, 11, 9], [10, 7, 7, 17], [10, 3, 13, 5], [9, 11, 10, 12], [12, 16, 8, 8], [17, 9, 5, 10], [16, 5, 15, 17], [15, 17, 15, 11], [7, 3, 4, 12], [6, 3, 3, 16], [12, 11, 16, 6], [15, 5, 3, 13], [9, 14, 11, 3], [13, 16, 4, 10], [8, 13, 15, 4], [11, 16, 9, 12], [11, 11, 7, 11], [7, 6, 4, 9], [9,

Traceback (most recent call last):
  File "C:\Users\hmoreira\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\hmoreira\AppData\Local\Temp\ipykernel_3508\2452764436.py", line 1, in <module>
    geneticAlgorithm(500, 100, 0.10, 5, 0.01)
  File "C:\Users\hmoreira\AppData\Local\Temp\ipykernel_3508\3811472864.py", line 26, in geneticAlgorithm
    mutation(finalPopulation, taxMutation, changeGene)
  File "C:\Users\hmoreira\AppData\Local\Temp\ipykernel_3508\4154055711.py", line -1, in mutation
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\hmoreira\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hmo

In [36]:
runNTimesGeneticAlgorithm(1)

44
36
0.06
5
0.19
4160
[[3, 10, 12, 11], [7, 13, 12, 16], [6, 16, 7, 10], [16, 17, 14, 12], [5, 3, 10, 17], [11, 4, 15, 3], [7, 11, 9, 9], [9, 10, 15, 10], [12, 4, 10, 14], [6, 11, 10, 14], [12, 13, 11, 10], [6, 14, 6, 7], [6, 16, 15, 12], [11, 3, 11, 12], [12, 15, 16, 8], [9, 10, 12, 10], [13, 4, 15, 7], [3, 3, 17, 10], [17, 16, 6, 11], [10, 12, 6, 16], [15, 16, 7, 13], [14, 7, 14, 3], [6, 12, 13, 7], [11, 16, 17, 16], [17, 14, 13, 10], [17, 9, 7, 15], [7, 12, 5, 5], [8, 14, 15, 15], [12, 16, 8, 9], [9, 6, 3, 15], [16, 16, 16, 10], [4, 6, 11, 7], [4, 3, 5, 6], [3, 7, 15, 12], [4, 9, 13, 6], [7, 5, 8, 6]]
Tamanho final Pop Aux:  2
Tamanho da final Pop: 2 Numero Element no cross: 2
After Fitness: 2 Pop Len: 34
After crossover: 36 Pop Len: 0
After mutation: 36 Pop Len: 0
Tamanho final Pop Aux:  2
Tamanho da final Pop: 2 Numero Element no cross: 2
After Fitness: 2 Pop Len: 34


KeyboardInterrupt: 